In [12]:
import requests
import pymysql
import base64
import pickle
import time
from datetime import datetime
from bs4 import BeautifulSoup

In [15]:
with open("pw.pkl", "wb") as f:
    pickle.dump(base64.b64encode(input("pw:").encode('utf-8')), f)

In [19]:
with open("pw.pkl", "rb") as f:
    pw = base64.b64decode(pickle.load(f)).decode('utf-8')

In [6]:
ts = time.time()
dt = datetime.fromtimestamp(ts).strftime("%Y-%m-%d")

signs_dict = {"水瓶":"Aquarius", "雙魚":"Pisces", "牡羊":"Aries", 
            "金牛":"Taurus", "雙子":"Gemini", "巨蟹":"Cancer", 
            "獅子":"Leo", "處女":"Virgo", "天秤":"Libra", 
            "天蠍":"Scorpio", "射手":"Sagittarius", "摩羯":"Capricorn"}

In [4]:
def get_all_holoscopes():
    # my_sign = input("請輸入你的星座（兩個字)：")
    url_head = "http://www.daily-zodiac.com/zodiac/"
    htmls = []

    for sign in signs_dict.values(): 
        url = url_head + sign
        # print(url)
        
        html = requests.get(url).text
        htmls.append(html)

    # print(len(htmls))

    soups = []
    for html in htmls:
        soup = BeautifulSoup(html, 'html.parser')
        soups.append(soup)

    holoscopes = []
    for soup in soups:
        holoscope = soup.find_all("meta", property="og:description")[0]['content']
        # print(trend)
        holoscopes.append(holoscope)

    all_holoscopes = dict(zip(signs_dict.keys(), holoscopes))
    return all_holoscopes

In [25]:
dt

'2022-06-05'

In [7]:
all_holoscopes = get_all_holoscopes()

In [8]:
list(all_holoscopes.keys())

['水瓶', '雙魚', '牡羊', '金牛', '雙子', '巨蟹', '獅子', '處女', '天秤', '天蠍', '射手', '摩羯']

In [31]:
data = tuple(i + tuple([dt]) for i in all_holoscopes.items())

In [32]:
data

(('水瓶',
  '週末在環境或身體上都適合做清理的動作，可以讓你達到清靜的效果。感情部分建議和另一半聊內心的心結，進而增進彼此的關係。幸運色是黃色。',
  '2022-06-05'),
 ('雙魚',
  '週末會有加班或出乎意料的拖延情形，很多事最後一刻才會有結果，請放輕鬆，心情七上八下對事情也沒有幫助。感情方面很多事情傾向於多幫助、多配合對方，相信他會感受到你的誠意。幸運色是黃色。',
  '2022-06-05'),
 ('牡羊',
  '週末有些狀況會扭轉，例如：讓你放心的事情會生變，建議要提高警覺才好。感情方面會遇到舊情人或想到有關於他的事情，回憶湧上心頭讓你有所感觸。幸運色是紫色。',
  '2022-06-05'),
 ('金牛',
  '週末應酬很多，而你卻沒有心情參與且覺得勞累，因此事後要好好休息才行。感情部分溝通時會被另一半抓到語病，建議千萬不要撒謊。幸運色是藍色。',
  '2022-06-05'),
 ('雙子',
  '週末會有欲速則不達的狀況，建議要多一點耐性，才能讓事情順利進行。感情方面你會收到與前任相關的消息，例如：看到對方的訊息；不妨當作回顧就好。幸運色是紫色。',
  '2022-06-05'),
 ('巨蟹',
  '週末有一些責任義務在身上，因此讓你過得很忙碌。感情部分對於金錢的用途，另一半無法認同，建議雙方要好好溝通才行。幸運色是粉紅色。',
  '2022-06-05'),
 ('獅子',
  '週末提案很多使你不知道參與哪個，也許很多朋友邀約必須篩選，或很多場合有人獻殷勤不知如何應對，使你精神勞累。感情方面桃花運旺，所到之處容易吸引人目光，多參加聚會有機會認識不錯的人。幸運色是金色。',
  '2022-06-05'),
 ('處女',
  '週末有很多計畫都讓你感興趣，建議當決定某一個提案時，就不要有遺憾的心。感情方面正處於要不要接受對方的階段，進而也在挑戰別人的耐性。幸運色是藍色。',
  '2022-06-05'),
 ('天秤',
  '週末適合清理、清掃，例如：整理環境或剪頭髮、護膚…等，不妨讓自己和環境都煥然一新吧！感情方面彼此都喜歡輕鬆的相處模式，建議選擇靜態活動比較好。幸運色是橘色。',
  '2022-06-05'),
 ('天蠍',
  '週末很多事情本來是幫忙的性質，最後變成了義務，導致假期會

In [9]:
def upload_data(data):
    with open("pw.pkl", "rb") as f:
        pw = base64.b64decode(pickle.load(f)).decode('utf-8')
    
    db = pymysql.connect(host="localhost",user="root",passwd=pw ,database="holoscopes")
    cursor = db.cursor()
    
    sql = "INSERT INTO daily_holoscopes(sign, holoscopes, date, weekday) \
        VALUES (%s, %s, %s, %s)"

    try:
        cursor.executemany(sql, data)
        db.commit()

    except Exception as e:
        print(e)
        db.rollback()

    finally:
        db.close()